In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [14]:
import shutil
import os.path
import requests
import tempfile
from pegaofg import *
from selenium import webdriver
from pdf2image import *
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from time import sleep

pasta = 'C:/Users/pdv/Desktop/selenium-imgs'

profile=webdriver.FirefoxOptions()
profile.set_preference("browser.download.folderList",2)
profile.set_preference("browser.download.manager.showWhenStarting",False)
profile.set_preference("browser.download.dir",pasta)
profile.set_preference("browser.helperApps.neverAsk.saveToDisk","application/octet-stream")
profile.add_argument("--window-size=1920,1080")
profile.add_argument('--disable-dev-shm-usage')
profile.add_argument('--disable-gpu')
profile.add_argument('--headless')
browser=webdriver.Firefox(options=profile)


dicionario = {
        'Dalbem':'https://www.supermercadosdalben.com.br/ofertas.html',
        'Fartura':'https://www.hortifrutifartura.com.br/wp-content/themes/fartura/images/banner-oferta.png',
#         'Goodbom':'https://www.goodbom.com.br/tabloides/index.php/goodbom-taquaral/',
        'Oba':'https://oba.digital/ofertas',
        'PagueMenos':'https://www.superpaguemenos.com.br/tabloide-digital/s',}


def zerapasta(pasta):
#     shutil.rmtree(pasta)
    for raiz, subpastas, arquivos in os.walk(pasta):                                      
        for arquivo in arquivos:
            os.unlink(f'{raiz}/{arquivo}')
        for subpasta in subpastas:
            os.rmdir(f'{raiz}/{subpasta}')
            
            
def checapasta(pasta):
    for raiz, subpastas, arquivos in os.walk(pasta):  
        feito = f'{pasta}feito/'
        if 'feito' in subpastas:
            zerapasta(feito)
        else:
            os.mkdir(feito)
        return feito


def pdf(mercado):
    pasta_mercado = f'C:/Users/pdv/Desktop/selenium-imgs/{mercado}/'
    saida = checapasta(pasta_mercado) #Devolve a pasta_mercado + /feito/
    contador = 0
    for _, _, arquivos in os.walk(pasta_mercado):
        print(arquivos)
        for arquivo in arquivos:
            print('arq', arquivo)
            if 'pdf' in arquivo:
                path = os.path.join(pasta_mercado,arquivo)
                print(path)
                if os.path.getsize(path) < 10000000:
                    saidapdf = os.path.join(saida,arquivo)
                    shutil.move(path, saidapdf)
                    continue
                images = convert_from_path(path,
                                    poppler_path=r'C:\Program Files (x86)\poppler-22.01.0\Library\bin',
    #                                 output_folder=saida,
                                    fmt='JPEG')
                for image in images:
                    image.save(f'{saida}{contador}.jpeg', 'JPEG')
                    contador += 1
                print(contador,' imagens salvas')
        break

def pegaof(dicionario, pasta):
    for mercado,site in dicionario.items():
        sleep(2)
        print(f'{mercado} : {site}')
        browser.get(site)
        pasta_mercado = f'{pasta}/{mercado}'
        listalink = []

        if mercado == 'Dalbem':
            lista = []
            az = browser.find_elements(By.TAG_NAME,'a')
            for a in az:
                if a.get_attribute('onclick'):
                    listalink.append(a.get_attribute('onclick'))
            for i in set(listalink):
                try:
                    lista.append(i.split("'")[1])                
                    lista.append(i.split('"')[1])
                except IndexError:
                    pass
            for i in lista:
                print(i)
                response = requests.get(i, stream=True)
                caminho = f'{pasta}/Dalbem/pdf{lista.index(i)}.pdf'
                with open(caminho, 'wb') as out_file:
                    shutil.copyfileobj(response.raw, out_file)
                del response

        if mercado == 'Fartura':
            url = 'https://www.hortifrutifartura.com.br/wp-content/themes/fartura/images/banner-oferta.png'
            response = requests.get(url, stream=True)
            if not response:
                continue
            else:
                with open(f'{pasta}/Fartura/img.png', 'wb') as out_file:
                    shutil.copyfileobj(response.raw, out_file)
            del response

        if mercado == 'Oba':
            contador = 0
            btn = browser.find_element(By.XPATH, '//a[contains(@href,"index.php?regiao=sp")]')
            btn.click()
            listas = []
            listas.append(browser.find_elements(By.TAG_NAME,'img'))
            listas.append(browser.find_elements(By.CLASS_NAME,'item active'))
            for lista in listas:
                for elem in lista:
                    if 'assets' not in str(elem.get_attribute('src')):
                        link = elem.get_attribute('src')
                        print(link)
                        response = requests.get(link, stream=True)
                        with open(f'{pasta}/oba/oba{contador}.jpeg', 'wb') as out_file:
                            shutil.copyfileobj(response.raw, out_file)
                        del response
                        contador += 1

        if mercado == 'PagueMenos':
            xs = WebDriverWait(browser, 20).until(EC.visibility_of_all_elements_located((By.ID, "internal")))
            # test = browser.find_elements(By.CLASS_NAME, " pt-br ")
            contador = 0
            for x in xs:
                teste = x.find_elements(By.CSS_SELECTOR, 'div.col col-sm-4 col-xs-6 align-mobile')
                filho = x.find_element(By.CLASS_NAME, 'pr')
                neto = filho.find_element(By.XPATH, './/*')
                bisneto = neto.find_element(By.CLASS_NAME, 'container')
                tataraneto = bisneto.find_element(By.CLASS_NAME, 'float')
                x2tataraneto = tataraneto.find_element(By.ID, 'tabloides')
                x3tataraneto = x2tataraneto.find_element(By.CLASS_NAME, "center")
                x4tataraneto = x3tataraneto.find_elements(By.TAG_NAME, "div")
                for x4 in x4tataraneto:
                    classe = x4.get_attribute('class')
                    if str(classe).startswith('col') and str(classe)[-6:] != 'hidden':
                        filhosdox4 = x4.find_elements(By.TAG_NAME, "div")
                        for f in filhosdox4:
                            netosdox4 = f.find_elements(By.TAG_NAME, "a")
                            for i in netosdox4:
                                response = requests.get(i.get_attribute('href'), stream=True)
                                with open(f'{pasta}/PagueMenos/pg{contador}.pdf', 'wb') as out_file:
                                    shutil.copyfileobj(response.raw, out_file)
                                del response
                                contador += 1
                                
        if mercado == 'Goodbom':
            pegaofg(pasta)


pegaof(dicionario,pasta)
pdf('Dalbem')

Dalbem : https://www.supermercadosdalben.com.br/ofertas.html
https://dalben.s3.sa-east-1.amazonaws.com/1650648922104833.pdf
https://dalben.s3.sa-east-1.amazonaws.com/1650377086463942.pdf
https://dalben.s3.sa-east-1.amazonaws.com/165064888764697.pdf
Fartura : https://www.hortifrutifartura.com.br/wp-content/themes/fartura/images/banner-oferta.png
Oba : https://oba.digital/ofertas
https://obahortifruti.com.br/ofertas/preco-verde/sp/1d.jpeg
https://obahortifruti.com.br/ofertas/preco-verde/sp/2d.jpeg
https://obahortifruti.com.br/ofertas/preco-verde/sp/3d.jpeg
https://obahortifruti.com.br/ofertas/preco-verde/sp/4d.jpeg
PagueMenos : https://www.superpaguemenos.com.br/tabloide-digital/s
['pdf0.pdf', 'pdf1.pdf', 'pdf2.pdf']
arq pdf0.pdf
C:/Users/pdv/Desktop/selenium-imgs/Dalbem/pdf0.pdf
arq pdf1.pdf
C:/Users/pdv/Desktop/selenium-imgs/Dalbem/pdf1.pdf
36  imagens salvas
arq pdf2.pdf
C:/Users/pdv/Desktop/selenium-imgs/Dalbem/pdf2.pdf
